In [103]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [104]:
people_df = pd.read_csv('people.csv')
act_train_df = pd.read_csv('act_train.csv')

In [105]:
people_df.head()
#people_df.columns

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [106]:
act_train_df.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


All variables are categorical, with the exception of 'char_38' in the people file, which is a continuous numerical variable.

People_df contains characteristics of each person. Act_train contains activities that were initiated, but not neccesarily completed. Each activity has information contains that id for the person, activity category, charactersitcs about that activity, as well as whether the activity was completed or not.

The goal is predict the outcome of the actitity -- either 0 or 1.





## Joining the dfs

You'll want to incorporate the characteristics of each person into your model? How would you do that?

Probably want to join on people_id -- so that each activity contains all the information about the person doing the activity and the activity.

First, rename char variables so they don't conflict with eachother.

Then drop date.

In [107]:
people_df = people_df.drop('date', axis=1)
act_train_df = act_train_df.drop('date', axis=1)

import re
p = re.compile('^char')

def renameCharColumns(colName, prpnd):
    if p.match(colName) != None:
        return prpnd + '_' + colName
    else:
        return colName

people_df.columns = [renameCharColumns(colName, prpnd='people') for colName in people_df.columns]
act_train_df.columns = [renameCharColumns(colName, prpnd='act') for colName in act_train_df.columns]

In [108]:
joined_df = act_train_df.merge(people_df, on='people_id', )

In [109]:
joined_df.head()


,people_id,activity_id,activity_category,act_char_1,act_char_2,act_char_3,act_char_4,act_char_5,act_char_6,act_char_7,...,people_char_29,people_char_30,people_char_31,people_char_32,people_char_33,people_char_34,people_char_35,people_char_36,people_char_37,people_char_38
0,ppl_100,act2_1734928,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36


Now to convert data types, clean the df, etc.

Actitivy category looks like it could be converted to a categorial variable. Many of the people_char features are booleans -- perhaps you should convert them to categorical vairables as well.




In [110]:
act_category_dummies = pd.get_dummies(joined_df['activity_category'], prefix='act_cat')
act_category_dummies.head()
joined_df = joined_df.join(act_category_dummies)
joined_df = joined_df.drop('activity_category', axis=1)
joined_df.head()

,people_id,activity_id,act_char_1,act_char_2,act_char_3,act_char_4,act_char_5,act_char_6,act_char_7,act_char_8,...,people_char_36,people_char_37,people_char_38,act_cat_type 1,act_cat_type 2,act_cat_type 3,act_cat_type 4,act_cat_type 5,act_cat_type 6,act_cat_type 7
0,ppl_100,act2_1734928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,ppl_100,act2_2434093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ppl_100,act2_3404049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ppl_100,act2_3651215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ppl_100,act2_4109017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [111]:
joined_df.head()

,people_id,activity_id,act_char_1,act_char_2,act_char_3,act_char_4,act_char_5,act_char_6,act_char_7,act_char_8,...,people_char_36,people_char_37,people_char_38,act_cat_type 1,act_cat_type 2,act_cat_type 3,act_cat_type 4,act_cat_type 5,act_cat_type 6,act_cat_type 7
0,ppl_100,act2_1734928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,ppl_100,act2_2434093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ppl_100,act2_3404049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ppl_100,act2_3651215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ppl_100,act2_4109017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [115]:
joined_df['people_char_2'].head()

0    type 2
1    type 2
2    type 2
3    type 2
4    type 2
Name: people_char_2, dtype: object

In [119]:
colsToEncode = ['people_char_1', 'people_char_2', 'people_char_3', 'people_char_4', 'people_char_5','people_char_6', 'people_char_7', 'people_char_8', 'people_char_9']

def oneHotEncode(colName, prefix, df):
    copy = df.copy()
    print(copy.head())
    dummies = pd.get_dummies(copy[colName], prefix=colName)
    print(dummies.head())
    copy = pd.concat([copy,dummies], axis=1)
    copy = copy.drop(colName, axis=1)
    return copy

joined_copy = joined_df.copy()
for _, col in enumerate(colsToEncode):
    joined_copy = oneHotEncode(col, 'people_char', joined_df)
    
    
print(joined_copy.shape)
joined_copy.head()


    

  people_id   activity_id act_char_1 act_char_2 act_char_3 act_char_4  \
0   ppl_100  act2_1734928        NaN        NaN        NaN        NaN   
1   ppl_100  act2_2434093        NaN        NaN        NaN        NaN   
2   ppl_100  act2_3404049        NaN        NaN        NaN        NaN   
3   ppl_100  act2_3651215        NaN        NaN        NaN        NaN   
4   ppl_100  act2_4109017        NaN        NaN        NaN        NaN   

  act_char_5 act_char_6 act_char_7 act_char_8      ...       people_char_36  \
0        NaN        NaN        NaN        NaN      ...                 True   
1        NaN        NaN        NaN        NaN      ...                 True   
2        NaN        NaN        NaN        NaN      ...                 True   
3        NaN        NaN        NaN        NaN      ...                 True   
4        NaN        NaN        NaN        NaN      ...                 True   

  people_char_37  people_char_38 act_cat_type 1 act_cat_type 2 act_cat_type 3  \
0    

,people_id,activity_id,act_char_1,act_char_2,act_char_3,act_char_4,act_char_5,act_char_6,act_char_7,act_char_8,...,act_cat_type 7,people_char_9_type 1,people_char_9_type 2,people_char_9_type 3,people_char_9_type 4,people_char_9_type 5,people_char_9_type 6,people_char_9_type 7,people_char_9_type 8,people_char_9_type 9
0,ppl_100,act2_1734928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ppl_100,act2_2434093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ppl_100,act2_3404049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ppl_100,act2_3651215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ppl_100,act2_4109017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
joined_copy.to_csv('intermediate_clean.csv')